In [1]:
import pandas as pd
import numpy as np
import time
from alpha_vantage.timeseries import TimeSeries

# read SP500 components 
df_tickers = pd.read_csv("sp500_components.csv")
df_tickers = df_tickers.rename(columns={"Unnamed: 0": "comp_index"})

# creating a list of all tickers
melted_tickers = pd.melt(df_tickers, id_vars="comp_index", var_name="year", value_name="ticker")
tickers = melted_tickers["ticker"].unique().tolist()

# replace YOUR_API_KEY with your actual Alpha Vantage API key
ts = TimeSeries(key='V1PFJOMDHX76TFF8', output_format='pandas')

# download the adjusted close price for each ticker
data_frames = []
for ticker in tickers:
    
    print(ticker)
    
    try:
        data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
    
        # rename the column and select only the adjusted close price
        data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
    
        # create ticker column
        data["ticker"] = ticker
    
        # get log-returns
        data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
    
        # sort the index in ascending order
        data = data.sort_index()
    
        # add data to data_frames list 
        data_frames.append(data)

    except:

        # wait 1 minute before trying again
        time.sleep(60)

# concatenate the data frames into a single data frame
combined_data = pd.concat(data_frames, axis=0)

# print the first 5 rows of the data
print(combined_data.head())



AAL
AAMRQ
AAPL
ABI
ABS
ABT
ABX
ACKH
ACV
ADM
ADP
            Adj_Close ticker   log_rtn
date                                  
2005-09-27  18.192017    AAL -0.060320
2005-09-28  19.323127    AAL  0.014247
2005-09-29  19.049776    AAL -0.038821
2005-09-30  19.803849    AAL -0.023054
2005-10-03  20.265719    AAL -0.030236
